In [2]:
import pandas as pd
from collections import defaultdict
import random
import glob

In [3]:
allc_table = pd.read_csv('FULL.ALLC-TABLE.tsv', sep = '\t', header = None)

## select use_sample by cell number
## by three ages has it

In [4]:
sample_cell_numder = allc_table[1].value_counts()
min_cells = 50
sample_cell_numder = sample_cell_numder[sample_cell_numder>min_cells]

all_samlpes = sample_cell_numder.index.tolist()
df = pd.DataFrame({'sample':all_samlpes})
df['celltype'] = df['sample'].apply(lambda x:x.split('.')[0])
celltypes_counts  = df['celltype'].value_counts()
use_celltypes = celltypes_counts[celltypes_counts == 3].index.tolist()
use_celltypes.remove('L5-IT-CTX-Glut')
use_celltypes.remove('MB-MY-Glut-Sero')

In [6]:
use_samples = df[df['celltype'].isin(use_celltypes)]['sample'].to_list()
len(use_samples)

120

In [8]:
allc_paths_dir = '/ceph/gale-1/qzeng/amb_datasets'
kind = 'CellType.Age'
ready_samples = []
for path in glob.glob(f'{allc_paths_dir}/{kind}/*/*.CGN-Merge.allc.tsv.gz'):
    sample = path.split('/')[-2]
    ready_samples.append(sample)
len(ready_samples)

111

In [25]:
remerge_samples = []
for sample in use_samples:
    if sample not in ready_samples:
        remerge_samples.append(sample)

In [38]:
remerge_samples

['Astro-NN.8wk',
 'STR-D1-Gaba.18mo',
 'DG-Glut.9mo',
 'Microglia-NN.8wk',
 'STR-D2-Gaba.9mo',
 'L6-CT-CTX-Glut.8wk',
 'OPC-NN.8wk',
 'L23-IT-CTX-Glut.8wk',
 'STR-PAL-Chst9-Gaba.9mo',
 'L6-CT-CTX-Glut.9mo',
 'OPC-NN.18mo',
 'CA1-ProS-Glut.18mo',
 'MEA-Gaba.8wk',
 'Sst-Gaba.8wk',
 'CEA-Gaba.18mo',
 'Pvalb-Gaba.9mo',
 'Sst-Gaba.18mo',
 'PAG-Glut.9mo',
 'LA-BLA-BMA-PA-Glut.18mo',
 'LA-BLA-BMA-PA-Glut.9mo',
 'Lamp5-Gaba.9mo',
 'MEA-COAp-Glut.8wk',
 'Endo-NN.18mo',
 'SUB-ProS-Glut.18mo']

## donwsample

In [27]:
allc_table = allc_table[allc_table[1].isin(remerge_samples)]
allc_table.columns= ['allc_paths','samples']
category_counts = allc_table['samples'].value_counts()
categories_to_downsample = category_counts[category_counts > 1000].index

In [28]:
downsampled_rows = []

for category in categories_to_downsample:
    category_rows = allc_table[allc_table['samples'] == category]
    sampled_rows = category_rows.sample(n=1000, random_state=42)  
    downsampled_rows.append(sampled_rows)

downsampled_df = pd.concat(downsampled_rows)

In [29]:
un_downsampled_df = allc_table[~allc_table['samples'].isin(categories_to_downsample)]

In [30]:
use_allc_table = pd.concat([un_downsampled_df, downsampled_df])

In [31]:
check_sample_counts = use_allc_table['samples'].value_counts().sort_values(ascending=False) 

In [32]:
all_samles = check_sample_counts.index.tolist()

In [33]:
random.shuffle(all_samles)
group_size = 3

group_num = {}

for i in range(0, len(all_samles), group_size):
    group_name = f'group{i//3}'
    group = all_samles[i:i+group_size]
    group_num[group_name] = check_sample_counts.loc[group].sum()
    tmpdf=use_allc_table[use_allc_table['samples'].isin(group)]
    tmpdf.to_csv(f'ALLC_TABLE/{group_name}.tsv', sep = '\t', header = None, index = None)
    

In [34]:
group_num.keys()

dict_keys(['group0', 'group1', 'group2', 'group3', 'group4', 'group5', 'group6', 'group7'])

In [35]:
group_num

{'group0': 2044,
 'group1': 1984,
 'group2': 1042,
 'group3': 1169,
 'group4': 954,
 'group5': 1613,
 'group6': 2243,
 'group7': 1748}

In [37]:
! gsutil -m cp -r ALLC_TABLE gs://ecker-rachel-amb-datasets/tmp

Copying file://ALLC_TABLE/group0.tsv [Content-Type=text/tab-separated-values]...
Copying file://ALLC_TABLE/group1.tsv [Content-Type=text/tab-separated-values]...
Copying file://ALLC_TABLE/group2.tsv [Content-Type=text/tab-separated-values]...
Copying file://ALLC_TABLE/group3.tsv [Content-Type=text/tab-separated-values]...
Copying file://ALLC_TABLE/group4.tsv [Content-Type=text/tab-separated-values]...
Copying file://ALLC_TABLE/group5.tsv [Content-Type=text/tab-separated-values]...
Copying file://ALLC_TABLE/group6.tsv [Content-Type=text/tab-separated-values]...
Copying file://ALLC_TABLE/group7.tsv [Content-Type=text/tab-separated-values]...
/ [8/8 files][  1.7 MiB/  1.7 MiB] 100% Done                                    
Operation completed over 8 objects/1.7 MiB.                                      
